# Tutorial 4: Agents in LangChain

In this tutorial, we'll explore Agents in LangChain, which are autonomous entities capable of using tools and making decisions to accomplish tasks.

In [ ]:
import os
from dotenv import load_dotenv
from langchain.llms import Groq
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.tools import Tool
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import create_sync_playwright_browser

# Load environment variables
load_dotenv()

# Initialize Groq LLM
llm = Groq(model_name="llama2-70b-4096")

## 1. Understanding Agent Architecture

Agents in LangChain combine an LLM with a set of tools to perform tasks. The LLM decides which tools to use and how to interpret their outputs.

## 2. Exploring Different Types of Agents

### Zero-shot React Agent

In [ ]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

zero_shot_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

zero_shot_agent.run(
    "What is the square root of the year Pythagoras was born? Round to the nearest integer."
)

### Conversational Agent

In [ ]:
conversational_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

conversational_agent.run(
    "Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it."
)

### Plan-and-Execute Agent

In [ ]:
plan_and_execute_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.PLAN_AND_EXECUTE,
    verbose=True
)

plan_and_execute_agent.run(
    "Find out who invented the telephone and calculate the number of years between that invention and the first moon landing."
)

## 3. Creating Custom Tools for Agents

In [ ]:
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

word_length_tool = Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)

custom_tools = [word_length_tool] + tools

custom_agent = initialize_agent(
    custom_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

custom_agent.run(
    "What's the length of the word 'supercalifragilisticexpialidocious' and its square root?"
)

## 4. Implementing a Multi-tool Agent for Task Solving

In [ ]:
# Create a synchronous browser
sync_browser = create_sync_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(sync_browser=sync_browser)
browser_tools = browser_toolkit.get_tools()

# Combine all tools
all_tools = tools + browser_tools + [word_length_tool]

multi_tool_agent = initialize_agent(
    all_tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

multi_tool_agent.run(
    """Please perform the following tasks:
    1. Find the current temperature in New York City.
    2. Calculate the square root of that temperature.
    3. Find a famous quote about weather and tell me how many characters it has.
    Provide a summary of your findings.
    """
)

# Close the browser
sync_browser.close()

## Conclusion

In this tutorial, we've explored Agents in LangChain, including different types of agents, creating custom tools, and implementing a multi-tool agent for complex task solving. Agents provide a powerful way to create autonomous systems that can leverage various tools and make decisions to accomplish tasks.